In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

recipe = pd.read_csv("/Users/apple/Downloads/recipe_dataset.csv")
print("The shape of recipe dataset: ", recipe.shape)
print("The recipe dataset information:")
print(recipe.info())

In [ ]:
recipe["Meal_Type"].value_counts()

In [ ]:
#drop null value
recipe.dropna(axis=0, how="any", inplace=True)

#create a new colume for gaining the value in the calorie colume
recipe["Calorie_num"]=recipe["Calorie"].map(lambda x:int(x.split(" ")[0]))

#delete the rows in which the calorie is 0
recipe = recipe[-recipe.Calorie_num.isin([0])]

#delete the duplicates
recipe.drop_duplicates(subset=["Name"], keep="first", inplace=True)

In [ ]:
print("The new shape of recipe dataset: ", recipe.shape)
print("The new recipe dataset information:")
print(recipe.info())

In [ ]:
recipe["Meal_Type"].value_counts()

In [ ]:
recipe = recipe.reset_index(drop=True)

In [ ]:
#find the closet row
def find_close(arr, value):
    index = (np.abs(arr-value)).argmin()        
    return index

In [ ]:
#print the recommendation list
def print_list(arr1, arr2, arr3, calorie, min_calorie, meal_type):
    if meal_type == 0:
        while calorie >= min_calorie:
            index = find_close(arr1,calorie)
            calorie = calorie-arr2["Calorie_num"][index]
            for key in arr3:
                arr3[key].append(arr2[key][index])
        return arr3
    elif meal_type == 1:
        while calorie >= min_calorie:
            index = find_close(arr1,calorie)
            calorie = calorie-arr2["Calorie_num"][3034+index]
            for key in arr3:
                arr3[key].append(arr2[key][3034+index])
            return arr3
    elif meal_type == 2:
        while calorie >= min_calorie:
            index = find_close(arr1,calorie)
            calorie = calorie-arr2["Calorie_num"][3962+index]
            for key in arr3:
                arr3[key].append(arr2[key][3962+index])
        return arr3
    elif meal_type == 3:
        while calorie >= min_calorie:
            index = find_close(arr1,calorie)
            calorie = calorie-arr2["Calorie_num"][5155+index]
            for key in arr3:
                arr3[key].append(arr2[key][5155+index])
        return arr3

In [ ]:
def cal_calorie_left(dic,calorie):
    num = 0
    for i in dic["Calorie_num"]:
        num = num + i
    return calorie-num

In [ ]:
import datetime

def recipe_rec(calorie, meal_num):
    recipe = pd.read_csv("/Users/apple/Downloads/recipe_dataset.csv")
    recipe.dropna(axis=0, how="any", inplace=True)
    recipe["Calorie_num"]=recipe["Calorie"].map(lambda x:int(x.split(" ")[0]))
    recipe.drop_duplicates(subset=["Name"], keep="first", inplace=True)
    recipe = recipe.reset_index(drop=True)
    
    #the output for each meal type
    breakfast_rec={"Name":[],"Ingredients_list":[],"Directions_list":[],"Prep_time":[],"Cook_time":[],"Calorie_num":[],"img_urls":[],"Meal_Type":[]}
    lunch_rec={"Name":[],"Ingredients_list":[],"Directions_list":[],"Prep_time":[],"Cook_time":[],"Calorie_num":[],"img_urls":[],"Meal_Type":[]}
    dinner_rec={"Name":[],"Ingredients_list":[],"Directions_list":[],"Prep_time":[],"Cook_time":[],"Calorie_num":[],"img_urls":[],"Meal_Type":[]}
    dessert_rec={"Name":[],"Ingredients_list":[],"Directions_list":[],"Prep_time":[],"Cook_time":[],"Calorie_num":[],"img_urls":[],"Meal_Type":[]}

    #array for each meal type
    arr_breakfast = recipe["Calorie_num"][recipe["Meal_Type"]=="breakfast"]
    arr_lunch = recipe["Calorie_num"][recipe["Meal_Type"]=="lunch"]
    arr_dinner = recipe["Calorie_num"][recipe["Meal_Type"]=="dinner"]
    arr_dessert = recipe["Calorie_num"][recipe["Meal_Type"]=="dessert"]
    
    #mininal calorie for each meal type
    min_breakfast = min(recipe.loc[recipe["Meal_Type"]=="breakfast"]["Calorie_num"])  
    min_lunch = min(recipe.loc[recipe["Meal_Type"]=="lunch"]["Calorie_num"])
    min_dinner = min(recipe.loc[recipe["Meal_Type"]=="dinner"]["Calorie_num"])
    min_dessert = min(recipe.loc[recipe["Meal_Type"]=="dessert"]["Calorie_num"])
    
    #get the current time
    dt = datetime.datetime.now()
    
    #limit the number of meal in 1 to 4, and the input calories has to be over 0
    if calorie >= 0:    
        if meal_num == 1:
            #default meal type is breakfast
            meal_type = input("Preferred meal-breakfast,lunch,dinner,dessert? ")
            if meal_type=="breakfast":
                print(print_list(arr_breakfast,recipe,breakfast_rec,calorie,min_breakfast,0))
            elif meal_type=="lunch":
                print(print_list(arr_lunch,recipe,lunch_rec,calorie,min_lunch,1))
            elif meal_type=="dinner":
                print(print_list(arr_dinner,recipe,dinner_rec,calorie,min_dinner,2))
            elif meal_type=="dessert":
                print(print_list(arr_dessert,recipe,dessert_rec,calorie,min_dessert,3))
            else:
                print(print_list(arr_breakfast,recipe,breakfast_rec,calorie,min_breakfast,0))
                
        elif meal_num == 2:
            #default meal types are breakfast and lunch
            meal_types = input("Preferred meal combination-breakfast&lunch, breakfast&dinner, breakfast&dessert, lunch&dinner, lunch&dessert, dinner&dessert?")
            if meal_types=="breakfast&lunch":
                bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.5,min_breakfast,0)                
                calorie_left = cal_calorie_left(bf_rec,calorie)
                print(bf_rec)               
                print(print_list(arr_lunch,recipe,lunch_rec,calorie_left,min_lunch,1))
            elif meal_types=="breakfast&dinner":
                bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.5,min_breakfast,0)
                calorie_left = cal_calorie_left(bf_rec,calorie)
                print(bf_rec)
                print(print_list(arr_dinner,recipe,dinner_rec,calorie_left,min_dinner,2))
            elif meal_types=="breakfast&dessert":
                bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.8,min_breakfast,0)
                calorie_left = cal_calorie_left(bf_rec,calorie)
                print(bf_rec)
                print(print_list(arr_dessert,recipe,dessert_rec,calorie_left,min_dessert,3))
            elif meal_types=="lunch&dinner":
                lun_rec = print_list(arr_lunch,recipe,lunch_rec,calorie*0.5,min_lunch,1)
                calorie_left = cal_calorie_left(lun_rec,calorie)
                print(lun_rec)
                print(print_list(arr_dinner,recipe,dinner_rec,calorie_left,min_dinner,2))
            elif meal_types=="lunch&dessert":
                lun_rec = print_list(arr_lunch,recipe,lunch_rec,calorie*0.8,min_lunch,1)
                calorie_left = cal_calorie_left(lun_rec,calorie)
                print(lun_rec)
                print(print_list(arr_dessert,recipe,dessert_rec,calorie_left,min_dessert,3))
            elif meal_types=="dinner&dessert":
                din_rec = print_list(arr_dinner,recipe,dinner_rec,calorie*0.8,min_dinner,2)
                calorie_left = cal_calorie_left(din_rec,calorie)
                print(din_rec)
                print(print_list(arr_dessert,recipe,dessert_rec,calorie_left,min_dessert,3))
            else:
                bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.5,min_breakfast,0)
                calorie_left = cal_calorie_left(bf_rec,calorie)
                print(bf_rec)
                print(print_list(arr_lunch,recipe,lunch_rec,calorie_left,min_lunch,1))
                
        elif meal_num == 3:
            #default meal types are breakfast, lunch & dinner
            meal_types = input("Preferred meal combination-breakfast&lunch&dinner, breakfast&lunch&dessert, breakfast&dinner&dessert, lunch&dinner&dessert?")
            if meal_types=="breakfast&lunch&dinner":
                bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.35,min_breakfast,0)
                calorie_left = cal_calorie_left(bf_rec,calorie)
                lun_rec = print_list(arr_lunch,recipe,lunch_rec,calorie*0.35,min_lunch,1)
                calorie_left = cal_calorie_left(lun_rec,calorie_left)
                print(bf_rec)
                print(lun_rec)
                print(print_list(arr_dinner,recipe,dinner_rec,calorie_left,min_dinner,2))
            elif meal_types=="breakfast&lunch&dessert":
                bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.4,min_breakfast,0)
                calorie_left = cal_calorie_left(bf_rec,calorie)
                lun_rec = print_list(arr_lunch,recipe,lunch_rec,calorie*0.4,min_lunch,1)
                calorie_left = cal_calorie_left(lun_rec,calorie_left)
                print(bf_rec)
                print(lun_rec)
                print(print_list(arr_dessert,recipe,dessert_rec,calorie_left,min_dessert,3)) 
            elif meal_types=="breakfast&dinner&dessert":
                bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.4,min_breakfast,0)
                calorie_left = cal_calorie_left(bf_rec,calorie)
                din_rec = print_list(arr_dinner,recipe,dinner_rec,calorie*0.4,min_dinner,2)
                calorie_left = cal_calorie_left(din_rec,calorie_left)
                print(bf_rec)
                print(din_rec)
                print(print_list(arr_dessert,recipe,dessert_rec,calorie_left,min_dessert,3))
            elif meal_types=="lunch&dinner&dessert":
                lun_rec = print_list(arr_lunch,recipe,lunch_rec,calorie*0.4,min_lunch,1)
                calorie_left = cal_calorie_left(lun_rec,calorie)
                din_rec = print_list(arr_dinner,recipe,dinner_rec,calorie*0.4,min_dinner,2)
                calorie_left = cal_calorie_left(din_rec,calorie_left)
                print(lun_rec)
                print(din_rec)
                print(print_list(arr_dessert,recipe,dessert_rec,calorie_left,min_dessert,3))
            else:
                bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.35,min_breakfast,0)
                calorie_left = cal_calorie_left(bf_rec,calorie)
                lun_rec = print_list(arr_lunch,recipe,lunch_rec,calorie*0.35,min_lunch,1)
                calorie_left = cal_calorie_left(lun_rec,calorie_left)
                print(bf_rec)
                print(lun_rec)
                print(print_list(arr_dinner,recipe,dinner_rec,calorie_left,min_dinner,2))      
        
        elif meal_num == 4:
            bf_rec = print_list(arr_breakfast,recipe,breakfast_rec,calorie*0.3,min_breakfast,0)
            calorie_left = cal_calorie_left(bf_rec,calorie)
            lun_rec = print_list(arr_lunch,recipe,lunch_rec,calorie*0.3,min_lunch,1)
            calorie_left = cal_calorie_left(lun_rec,calorie_left)
            din_rec = print_list(arr_dinner,recipe,dinner_rec,calorie*0.3,min_dinner,2)
            calorie_left = cal_calorie_left(din_rec,calorie_left)
            print(bf_rec)
            print(lun_rec)
            print(din_rec)
            print(print_list(arr_dessert,recipe,dessert_rec,calorie_left,min_dessert,3))
        else:
            print("The number of meal has to be in 1 to 4.")
    else:
        print("The calorie number has be over 0")

In [ ]:
calorie = int(input("Please input a calorie intake: "))
meal_num = int(input("Please input the number of meal: "))
recipe_rec(calorie, meal_num)